In [1]:
#P0: CARGAMOS DRIVE y definimos donde tenemos el dataset

from google.colab import drive
drive.mount('/content/gdrive')

path='/content/gdrive/My Drive/movie_data.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Creamos una función para tokenizar y limpiar datos al tiempo

In [0]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Creamos una función para que lee y devuelve un documento a la vez

In [0]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [6]:
next(stream_docs(path))

('"All those that identify this as a simple rip-off of \'Elephant\' - are there no other comments that you can make towards the movie on its own merits as an individual film (regardless of its apparent similarities to other movies).<br /><br />All those that question the validity of the movie - in terms of its stereotypical characters (the obligatory gay, the jock, the disabled kid, etc) - I\'m not sure how long it has been since you were at school - but regardless of how amateurish the acting may be - the happenings that go one are surprisingly close to what may actually happen.<br /><br />And all those that disregard the film as being so simple: just six teenagers with the regular teenage angst that pushes one over the edge... did anyone stop to think, and take notice that the girl who took her life wasn\'t actually one of those six!! As one comment points out - she had screen time of maybe 2min max (excluding the final scenes). I think the point of the film is not only to make an is

Creamos una función que tomara una secuencia de documentos y devolverá un número particular de documentos

In [0]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [0]:
#p2: definimos una versión liviana de CountVectorizer+TfidfVectorizer llamada HashingVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

#definimos como algoritmo la regressión logistica en el decenso gradiante
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path)

In [18]:
conda install -c conda-forge pyprind

SyntaxError: ignored

In [22]:
import re
import numpy as np
##import pyprind
from nltk.corpus import stopwords
stop = stopwords.words('english')

pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

LookupError: ignored